In [1]:
import pandas as pd
import tweepy
import requests
from bs4 import BeautifulSoup
import re
from pathlib import Path
import json
import os

## La liste des rapports déjà présents

In [2]:
old_reports_df = pd.read_csv('./rapports-senat.csv', sep=';')
old_reports_df

,full_id,title,desc,pdf_url
0,r21-805,Accompagner la rénovation de la presse quotidi...,"Rapport d'information de M. Michel LAUGIER, fa...",http://www.senat.fr/rap/r21-805/r21-8051.pdf
1,r21-800,"La politique de la ville, un tremplin pour les...",Rapport d'information de Mmes Viviane ARTIGALA...,http://www.senat.fr/rap/r21-800/r21-8001.pdf
2,r21-799,Une inflation tirée par les matières premières...,Rapport d'information de M. Daniel GREMILLET e...,http://www.senat.fr/rap/r21-799/r21-7991.pdf
3,r21-777,Quel avenir pour le corps diplomatique ?,Rapport d'information de MM. Jean-Pierre GRAND...,http://www.senat.fr/rap/r21-777/r21-7771.pdf
4,r21-776,Finale de la Ligue des Champions au Stade de F...,Rapport d'information de MM. François-Noël BUF...,http://www.senat.fr/rap/r21-776/r21-7761.pdf
...,...,...,...,...
149,r21-801,Nucléaire et hydrogène : l'urgence d'agir,"Rapport d'information de MM. Daniel GREMILLET,...",http://www.senat.fr/rap/r21-801/r21-8011.pdf
150,r21-882,Sur la sécurisation de la chasse,"Rapport d'information de M. Patrick CHAIZE, fa...",http://www.senat.fr/rap/r21-882/r21-8821.pdf
151,r21-885,Prévenir la délinquance des mineurs - Éviter l...,Rapport d'information de Mme Céline BOULAY-ESP...,NaN
152,r21-905,Compétitivité de la ferme France,"Rapport d'information de MM. Laurent DUPLOMB, ...",http://www.senat.fr/rap/r21-905/r21-9051.pdf


## Quels sont les nouvaux rapports d'information publiés par le Sénat ? 

Pour le savoir, on a besoin de récupérer la liste sur `http://www.senat.fr/rapports/rapports-information.html`.  
On va utiliser `bs4` pour parser le HTML.

- On cherche tous les liens `/notice-rapport/*`

In [3]:
r = requests.get('http://www.senat.fr/rapports/rapports-information.html')

In [4]:
soup = BeautifulSoup(r.text, 'html.parser')

In [5]:
new_reports_urls = []
for a in soup.find(id='encours').parent.find_all(lambda tag: tag.name == 'a' and tag.get('href').startswith('/notice-rapport')):
    [id] = re.findall('notice-rapport\/\d+\/(.+)-notice\.html', a.get('href'))
    if id not in old_reports_df['full_id'].values:
        new_reports_urls.append(a.get('href'))
        

In [6]:
new_reports_df = pd.DataFrame()
for href in new_reports_urls:
    # id
    [full_id] = re.findall('notice-rapport\/\d+\/(.+)-notice\.html', href)
    # retrieve and parse page
    r = requests.get(f'http://www.senat.fr{href}')
    soup = BeautifulSoup(r.text, 'html.parser')
    # we need:
    # - title
    title = soup.find('h1').text
    # - desc 
    desc = soup.find('h2', {'class': 'subtitle-01'}).text.strip().split('\n')[0]
    # - pdf url
    try:
        pdf_url = soup.find(lambda tag: tag.name == 'a' and tag.get_text() == 'Le rapport au format pdf').get('href')
        pdf_url = f'http://www.senat.fr{pdf_url}'
    except Exception as e:
        print(e)
        pdf_url = ''

    new_reports_df = pd.concat([new_reports_df, pd.Series({
        'full_id': full_id,
        'title': title,
        'desc': desc,
        'pdf_url': pdf_url
    }).to_frame().T])

In [7]:
new_reports_df

""


## On sauvegarde les nouveaux rapport

In [8]:
(pd
    .concat([old_reports_df, new_reports_df], ignore_index=True)
    .drop_duplicates(subset=['full_id'])
    .to_csv('rapports-senat.csv', sep=';', index=False)
)

## On les récupère 

In [9]:
Path('../data/senat/').mkdir(parents=True, exist_ok=True)
for index, row in new_reports_df.iterrows():
    if row['pdf_url']:
        r = requests.get(row['pdf_url'])
        Path(f'../data/senat/{row["full_id"]}.pdf').write_bytes(r.content)

## On les tweete

In [10]:
try:
    credentials = json.load(open('../twitter-credentials.json'))
except:
    # gh actions secrets
    credentials = {key: os.environ[key] for key in ["TWITTER_API_KEY", "TWITTER_API_SECRET", "TWITTER_ACCESS_KEY", "TWITTER_ACCESS_SECRET"]}


In [11]:
auth = tweepy.OAuthHandler(credentials['TWITTER_API_KEY'], credentials['TWITTER_API_SECRET'])
auth.set_access_token(credentials['TWITTER_ACCESS_KEY'], credentials['TWITTER_ACCESS_SECRET'])
api = tweepy.API(auth)

try:
    api.verify_credentials()
    print("Authentication OK")
except Exception as e:
    print(e)
    print("Error during authentication")

Authentication OK


In [12]:
for index, row in new_reports_df.iterrows():
    tweet = f'ℹ️ • Nouveau rapport d\'information {row["pdf_url"]} {row["title"]}'
    # pour le momnet, on ne tweet que les rapports d'information et d'enquête
    if len(tweet) > 280:
        tweet = tweet[0:277] + '...'    
    print(tweet)
    api.update_status(tweet)